In [178]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)
import os


import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from callbacks import all_callbacks

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1

from qkeras.qlayers import QDense, QActivation
from qkeras.quantizers import quantized_bits, quantized_relu
import tensorflow.compat.v1 as tf1
from qkeras.utils import model_save_quantized_weights

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning


from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import random
import math


from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
from qkeras.utils import model_save_quantized_weights, load_qmodel
import hls4ml
import qkeras.utils

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mae
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score
import csv

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from statistics import mean

ModuleNotFoundError: No module named 'callbacks'

## Test the final results. Each directory contains 50 random models with 3 layers with precisions from 8 to 2 and RFs=1,2,3,4,10

In [213]:
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects
import qkeras.qtools.qtools_util
import qkeras.estimate
import estimator.estimator as est
import utils
import utils_model as utmo
from importlib import reload
import csv



In [10]:
prec = 8
reuse = 1
layer_num = 50
loaded_models = []
filename = filename = "/media/dmitsas/BOOT-REPAIR/final_testing_lut/prec_" + str(prec) + "/reuse_" + str(reuse) + "/model_"
for i in range(layer_num):
            co = {}
            _add_supported_quantized_objects(co)
            loaded_models.append(load_model(filename + str(i) + '/KERAS_check_best_mode.h5', custom_objects=co))

2024-10-12 21:41:01.518956: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [11]:
def recover_params(filename,layer_num):
    precision = []
    for i in range(0,layer_num,1):
        with open(filename + str(i) + "/precision.txt") as f:
            for lines in f:
                precision.append(int(lines))
    f.close()
    
    RF = []
    for i in range(0,layer_num,1):
        with open(filename + str(i) + "/reuse.txt") as f:
            for lines in f:
                RF.append(int(lines))
    f.close()
    
    selector = []
    for i in range(0,layer_num,1):
        with open(filename + str(i) + "/impl.txt") as f:
            for lines in f:
                selector.append(int(lines))
    f.close()
    return precision,RF,selector

In [12]:
prec = 8
reuse = 1
layer_num = 50
rf_range = [1,2,3,4,10]
filename = "/media/dmitsas/BOOT-REPAIR/final_testing_lut/prec_" + str(prec) + "/reuse_" + str(reuse) + "/model_"
x,y,z = recover_params(filename,layer_num)

In [7]:

#####Verify if the recovered parameters are correct

for prec in range(8,1,-1):
    for reuse in rf_range:
        filename = filename = "/media/dmitsas/BOOT-REPAIR/final_testing_lut/prec_" + str(prec) + "/reuse_" + str(reuse) + "/model_"
        x,y,z = recover_params(filename,layer_num)
        if x[0] !=prec:
            print("Correct precision: " + str(prec) + "correct reuse: " + str(reuse))
            print("Wrong precision: " + str(x[0]) + "wrong reuse: " + str(reuse))
        if y[0] != reuse:
            print("Correct reuse: " + str(reuse))
            print("Wrong reuse: " + str(y[0]))

In [214]:
reload(utils)
reload(est)


<module 'estimator.estimator' from '/home/dmitsas/Downloads/backup/testing/estimator_testing/estimator/estimator.py'>

In [215]:
%%time
from statistics import median
layer_num = 50
prec = 8
reuse = 1
dirname = "./final_results/results/prec_" + str(prec) + "/reuse_" + str(reuse) 
rf_range = [1,2,3,4,10]
#all_DSP = [[] for x in range(7)]
all_DSP = []
for prec in range(8,1,-1):
    for reuse in rf_range:
        filename = "/media/dmitsas/BOOT-REPAIR/final_testing/prec_" + str(prec) + "/reuse_" + str(reuse) + "/model_"
        res_name = "/media/dmitsas/BOOT-REPAIR/final_testing/prec_" + str(prec) + "/reuse_" + str(reuse)
        #precision,RF,selector = recover_params(filename,layer_num)
        #precision = np.zeros(layer_num).astype(int) + prec
        RF = np.zeros(layer_num).astype(int) + reuse
        selector = np.zeros(layer_num).astype(int)
        
        precision = recover_params(filename,layer_num)[0]
        RF = recover_params(filename,layer_num)[1]
        selector = recover_params(filename,layer_num)[2]
        FF = []
        LUT = []
        DSP = []
        FF_pred = []
        LUT_pred = []
        DSP_pred = []
        dif = []
        dif_FF = []
        dif_DSP = []
        loaded_models = []
        for i in range(layer_num):
            co = {}
            _add_supported_quantized_objects(co)
            loaded_models.append(load_model(filename + str(i) + '/KERAS_check_best_mode.h5', custom_objects=co))
            ff,lut = utils.utils.get_resources(filename + str(i),model = loaded_models[i])
            DSP.append(utils.utils.get_DSP())
            FF.append(ff)
            LUT.append(lut)
            dsp_pred,lut_pred,ff_pred = utils.utils.gather(loaded_models[i],precision[i],int(RF[i]),selector[i],False)
            FF_pred.append(ff_pred)
            LUT_pred.append(lut_pred)
            DSP_pred.append(dsp_pred)
        FF_pred = np.array(FF_pred)
        LUT_pred = np.array(LUT_pred)
        DSP_pred = np.array(DSP_pred)
        FF = np.array(FF)
        LUT = np.array(LUT)
        DSP = np.array(DSP)
        
        LUT_single = LUT.reshape(-1,1)
        LUT_single_pred = LUT_pred.reshape(-1,1)

        FF_single = FF.reshape(-1,1)
        FF_single_pred = FF_pred.reshape(-1,1)

        DSP_single = DSP.reshape(-1,1)
        DSP_single_pred = DSP_pred.reshape(-1,1)
            
        dif = (100*abs(LUT_single - LUT_single_pred)/LUT_single)
        dif_FF = (100*abs(FF_single - FF_single_pred)/FF_single)
        if selector[0]==1:
            dif_DSP=((100*abs(DSP_single - DSP_single_pred)/DSP_single))
        else:
            dif_DSP.append(0)
        LUT_error = []
        FF_error = []
        DSP_error = []
        with open(res_name + "/results_new.txt",'w') as f:
            for k in dif:
                for j in k:
                    if j < 40:
                        LUT_error.append(j)
            for z in dif_FF:
                for j in z:
                    if j < 40:
                        FF_error.append(j)
#             for l in dif_DSP:
#                 for j in l:
#                     if j< 10:
#                         DSP_error.append(j)
            LUT_error = np.array(LUT_error)
            FF_error = np.array(FF_error)
            #DSP_error = np.array(DSP_error)
            f.write("LUT_error: " + str(median(LUT_error)) + "\nFF_error: " + str(median(FF_error))) #+ "\nDSP_error: " + str((DSP_error.mean())))
        #all_DSP.append(DSP_error)
        #real_muls = reg_ff.regression.get_muls(loaded_models,layer_num)


... quantizing model
-----------------------------------------------------------
zero LUT elements: 31
Total LUTs: 5505
Zeros: 421
Total multiplication LUTs: 2887
Total accumulator LUTs: 2618
Total DSPs: 59
Total layer additions: 209
Total bias additions: 29
Number of repeated weights: 20
Total Flip Flops: 6964
Real multiplications: 239
... quantizing model
-----------------------------------------------------------
zero LUT elements: 61
Total LUTs: 10474
Zeros: 862
Total multiplication LUTs: 5117
Total accumulator LUTs: 5357
Total DSPs: 122
Total layer additions: 443
Total bias additions: 44
Number of repeated weights: 74
Total Flip Flops: 13407
Real multiplications: 488
... quantizing model
-----------------------------------------------------------
zero LUT elements: 60
Total LUTs: 9618
Zeros: 747
Total multiplication LUTs: 4965
Total accumulator LUTs: 4653
Total DSPs: 123
Total layer additions: 397
Total bias additions: 26
Number of repeated weights: 33


/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. 

Total Flip Flops: 11786
Real multiplications: 423


... quantizing model
-----------------------------------------------------------
zero LUT elements: 11
Total LUTs: 1359
Zeros: 214
Total multiplication LUTs: 644
Total accumulator LUTs: 715
Total DSPs: 21
Total layer additions: 26
Total bias additions: 29
Number of repeated weights: 3
Total Flip Flops: 1787
Real multiplications: 56
... quantizing model
-----------------------------------------------------------
zero LUT elements: 28
Total LUTs: 5105
Zeros: 857
Total multiplication LUTs: 2685
Total accumulator LUTs: 2420
Total DSPs: 66
Total layer additions: 187
Total bias additions: 33
Number of repeated weights: 13
Total Flip Flops: 6527
Real multiplications: 223
... quantizing model
-----------------------------------------------------------
zero LUT elements: 36
Total LUTs: 4915
Zeros: 828
Total multiplication LUTs: 2561
Total accumulator LUTs: 2354
Total DSPs: 53
Total layer additions: 187
Total bias additions: 27
Number of repeated weights: 11
Total Flip Flops: 6335
Real multiplic

/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. 

...

/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


-----------------------------------------------------------
zero LUT elements: 125
Total LUTs: 24999
Zeros: 581
Total multiplication LUTs: 12294
Total accumulator LUTs: 12705
Total DSPs: 281
Total layer additions: 1124
Total bias additions: 31
Number of repeated weights: 186
Total Flip Flops: 28171
Real multiplications: 1155


/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


... quantizing model
-----------------------------------------------------------
zero LUT elements: 11
Total LUTs: 1253
Zeros: 124
Total multiplication LUTs: 603
Total accumulator LUTs: 650
Total DSPs: 10
Total layer additions: 44
Total bias additions: 6
Number of repeated weights: 2
Total Flip Flops: 1639
Real multiplications: 51
... quantizing model
-----------------------------------------------------------
zero LUT elements: 11
Total LUTs: 2064
Zeros: 203
Total multiplication LUTs: 998
Total accumulator LUTs: 1066
Total DSPs: 26
Total layer additions: 43
Total bias additions: 39
Number of repeated weights: 6
Total Flip Flops: 2605
Real multiplications: 84
... quantizing model
-----------------------------------------------------------
zero LUT elements: 28
Total LUTs: 5603
Zeros: 581
Total multiplication LUTs: 2974
Total accumulator LUTs: 2629
Total DSPs: 62
Total layer additions: 219
Total bias additions: 20
Number of repeated weights: 12
Total Flip Flops: 6964
Real multiplication

/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/dmitsas/.local/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.4.2. 

In [218]:
rf_range = [1,2,3,4,10]
lut_error = []
ff_error = []
dsp_error = []
for prec in range(8,1,-1):
    for reuse in rf_range:
        res_name = "/media/dmitsas/BOOT-REPAIR/final_testing_lut/prec_" + str(prec) + "/reuse_" + str(reuse)
        with open("results_new.csv",'w') as csvfile:
            writer = csv.writer(csvfile)
        #print("precision: " + str(prec))
        #print("RF: " + str(reuse))
        with open(res_name + "/results_new.txt") as f:
            for i,lines in enumerate(f):
                #print(i)
                if i==1:
                    #lut_error.append(float(lines.split("\n")[0].split(":")[1]))#[0].split(" ")[1])
                    print("Precision = " + str(prec) + "\nreuse = " + str(reuse))
                    print(float(lines.split("\n")[0].split(":")[1]))
                    print("---------------------------------------")
#                 elif i==1:
#                     ff_error.append(float(lines.split("\n")[0].split(":")[1]))#[0].split(" ")[1])
#                     print("Precision = " + str(prec) + "\nreuse = " + str(reuse))
#                     print(float(lines.split("\n")[0].split(":")[1]))
#                     print("---------------------------------------")
#                 else:
#                     dsp_error.append(float(lines.split("\n")[0].split(":")[1]))#[0].split(" ")[1])
#                     print("Precision = " + str(prec) + "\nreuse = " + str(reuse))
#                     print(float(lines.split("\n")[0].split(":")[1]))
#                     print("---------------------------------------")
ff_error = np.array(ff_error)
lut_error = np.array(lut_error)
dsp_error = np.array(dsp_error)
rf_range = np.array(rf_range)             

Precision = 8
reuse = 1
14.636822470923853
---------------------------------------
Precision = 8
reuse = 2
17.53995715933432
---------------------------------------
Precision = 8
reuse = 3
17.545119038380285
---------------------------------------
Precision = 8
reuse = 4
14.598278335724533
---------------------------------------
Precision = 8
reuse = 10
12.951684937068615
---------------------------------------
Precision = 7
reuse = 1
13.67305751765893
---------------------------------------
Precision = 7
reuse = 2
18.07277672948115
---------------------------------------
Precision = 7
reuse = 3
16.398653702318622
---------------------------------------
Precision = 7
reuse = 4
13.324752775304415
---------------------------------------
Precision = 7
reuse = 10
16.32157810822381
---------------------------------------
Precision = 6
reuse = 1
15.27767285932561
---------------------------------------
Precision = 6
reuse = 2
20.445418035779483
---------------------------------------
Precisi

In [86]:
ff_error = ff_error.reshape(7,5)

In [87]:
lut_error = lut_error.reshape(7,5)
dsp_error = dsp_error.reshape(7,5)

In [88]:
lut_error

array([[ 6.47387175,  5.29263204,  4.99105546,  4.0373044 ,  8.77688572],
       [ 6.01632302,  7.10710711,  8.80955704,  7.28500355,  6.09194895],
       [ 6.41781499,  7.7416569 ,  8.22629724,  8.3668968 ,  5.69740044],
       [ 7.11382902,  8.13376991,  8.62629247,  6.52920962,  8.19648465],
       [ 4.7739399 , 10.54569966, 17.67870869, 14.71269353,  5.63196737],
       [ 5.67909773, 12.46297562, 18.05423857, 14.03163997,  6.67508418],
       [ 6.34209763, 13.14691623, 22.89410258, 25.98987109,  5.2550783 ]])

In [89]:
le = pd.DataFrame(lut_error)
le.index=(np.arange(8,1,-1))
ffe = pd.DataFrame(ff_error,)
ffe.index = le.index
dspdf = pd.DataFrame(dsp_error)
dspdf.index = le.index

In [90]:
LUT_pd = pd.DataFrame([["LUT_error",""]])
FF_pd = pd.DataFrame([["FF_error",""]])
RF_pd = pd.DataFrame(["RF"])
RF_pd.insert(0,'Empty',"")

In [91]:
#RF_pd.to_csv("./results_lut.csv",index_label="LUT_error",index=False)
LUT_pd.to_csv("./results_lut.csv",index = False,header=False)
RF_pd.to_csv("./results_lut.csv",mode="a",index=False,header=False)
le.to_csv("./results_lut.csv",mode='a',header=rf_range.astype(str),index_label="Precision",float_format='%.3f')

FF_pd.to_csv("./results_lut.csv",index = False,header=False,mode = 'a')
RF_pd.to_csv("./results_lut.csv",mode="a",index=False,header=False)
ffe.to_csv("./results_lut.csv",mode='a',header=rf_range.astype(str),index_label="Precision",float_format='%.3f')



### DSP handling

In [94]:
DSP_error.shape

(150,)